# Logistic Regression for Audio Classification: Speech vs Music

Welcome to your audio classification lab! You will build a logistic regression classifier to distinguish between speech and music. This assignment will step you through how to do this with a Neural Network mindset, adapting the image classification approach to audio data.

**Instructions:**
- Do not use loops (for/while) in your code, unless the instructions explicitly ask you to do so.
- Use `np.dot(X,Y)` to calculate dot products.

**You will learn to:**
- Build the general architecture of a learning algorithm, including:
    - Initializing parameters
    - Calculating the cost function and its gradient
    - Using an optimization algorithm (gradient descent) 
- Gather all three functions above into a main model function, in the right order.
- Work with audio data by converting it to mel-spectrograms

## Table of Contents
- [1 - Packages](#1)
- [2 - Overview of the Problem](#2)
    - [Exercise 1](#ex-1)
    - [Exercise 2](#ex-2)
- [3 - General Architecture of the learning algorithm](#3)
- [4 - Building the parts of our algorithm](#4)
    - [4.1 - Helper functions](#4-1)
        - [Exercise 3 - sigmoid](#ex-3)
    - [4.2 - Initializing parameters](#4-2)
        - [Exercise 4 - initialize_with_zeros](#ex-4)
    - [4.3 - Forward and Backward propagation](#4-3)
        - [Exercise 5 - propagate](#ex-5)
    - [4.4 - Optimization](#4-4)
        - [Exercise 6 - optimize](#ex-6)
        - [Exercise 7 - predict](#ex-7)
- [5 - Merge all functions into a model](#5)
    - [Exercise 8 - model](#ex-8)
- [6 - Further analysis](#6)
- [7 - Test with your own audio](#7)

<a name='1'></a>
## 1 - Packages

First, let's import all the packages that you will need during this assignment.
- [numpy](https://numpy.org/doc/1.20/) is the fundamental package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a library to plot graphs in Python.
- [librosa](https://librosa.org/) is a library for audio analysis.
- audio_utils contains custom functions to load and preprocess audio data.

In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import librosa
import librosa.display
from audio_utils import load_dataset
from public_tests import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

<a name='2'></a>
## 2 - Overview of the Problem

**Problem Statement**: You are given an audio dataset containing:
- A training set of m_train audio clips labeled as speech (y=1) or music (y=0)
- A test set of m_test audio clips labeled as speech or music
- Each audio clip is converted to a mel-spectrogram of shape (n_mels, time_steps)

You will build a simple audio classification algorithm that can correctly classify audio clips as speech or music.

Let's load the data by running the following code.

In [ ]:
# Loading the data (speech/music)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

We added "_orig" at the end of spectrogram datasets (train and test) because we are going to preprocess them. After preprocessing, we will end up with train_set_x and test_set_x (the labels train_set_y and test_set_y don't need any preprocessing).

Each line of your train_set_x_orig and test_set_x_orig is an array representing a mel-spectrogram. You can visualize an example by running the following code. Feel free to change the `index` value and re-run to see other spectrograms.

In [ ]:
# Example of a spectrogram
index = 0
plt.figure(figsize=(10, 4))
librosa.display.specshow(train_set_x_orig[index], x_axis='time', y_axis='mel', sr=22050, hop_length=512)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
print("y = " + str(train_set_y[:, index]) + ", it's a '" + classes[train_set_y[0, index]].decode("utf-8") + "' audio clip.")
plt.show()

Many software bugs in deep learning come from having matrix/vector dimensions that don't fit. If you can keep your matrix/vector dimensions straight you will go a long way toward eliminating many bugs.

<a name='ex-1'></a>
### Exercise 1
Find the values for:
- m_train (number of training examples)
- m_test (number of test examples)
- n_mels (number of mel frequency bands)
- time_steps (number of time steps in spectrogram)

Remember that `train_set_x_orig` is a numpy-array of shape (m_train, n_mels, time_steps). For instance, you can access `m_train` by writing `train_set_x_orig.shape[0]`.

In [ ]:
#(≈ 4 lines of code)
# m_train = 
# m_test = 
# n_mels = 
# time_steps = 
# YOUR CODE STARTS HERE

# YOUR CODE ENDS HERE

print("Number of training examples: m_train = " + str(m_train))
print("Number of testing examples: m_test = " + str(m_test))
print("Number of mel bands: n_mels = " + str(n_mels))
print("Number of time steps: time_steps = " + str(time_steps))
print("Each spectrogram is of shape: (" + str(n_mels) + ", " + str(time_steps) + ")")
print("train_set_x_orig shape: " + str(train_set_x_orig.shape))
print("train_set_y shape: " + str(train_set_y.shape))
print("test_set_x_orig shape: " + str(test_set_x_orig.shape))
print("test_set_y shape: " + str(test_set_y.shape))

For convenience, you should now reshape spectrograms of shape (n_mels, time_steps) into a numpy-array of shape (n_mels × time_steps, 1). After this, our training (and test) dataset is a numpy-array where each column represents a flattened spectrogram. There should be m_train (respectively m_test) columns.

<a name='ex-2'></a>
### Exercise 2
Reshape the training and test data sets so that spectrograms of size (n_mels, time_steps) are flattened into single vectors of shape (n_mels × time_steps, 1).

A trick when you want to flatten a matrix X of shape (a, b, c) to a matrix X_flatten of shape (b × c, a) is to use:
```python
X_flatten = X.reshape(X.shape[0], -1).T      # X.T is the transpose of X
```

In [ ]:
# Reshape the training and test examples
#(≈ 2 lines of code)
# train_set_x_flatten = ...
# test_set_x_flatten = ...
# YOUR CODE STARTS HERE

# YOUR CODE ENDS HERE

print("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print("train_set_y shape: " + str(train_set_y.shape))
print("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print("test_set_y shape: " + str(test_set_y.shape))

To represent spectrograms, the values are already normalized between 0 and 1 during the mel-spectrogram conversion process.

One common preprocessing step in machine learning is to center and standardize your dataset. However, for spectrograms (like images), the data is already in a suitable range [0, 1].

Let's verify our dataset is ready:

In [ ]:
train_set_x = train_set_x_flatten
test_set_x = test_set_x_flatten

print("Data range: min =", train_set_x.min(), ", max =", train_set_x.max())

<font color='cyan'>

**What you need to remember:**

Common steps for pre-processing an audio dataset:
- Convert audio to mel-spectrograms (time-frequency representation)
- Figure out the dimensions and shapes of the problem (m_train, m_test, n_mels, time_steps, ...)
- Reshape the datasets such that each example is now a vector of size (n_mels × time_steps, 1)
- Ensure data is normalized (spectrograms are already in [0, 1] range)

<a name='3'></a>
## 3 - General Architecture of the learning algorithm

It's time to design a simple algorithm to distinguish speech from music.

You will build a Logistic Regression, using a Neural Network mindset. The following Figure explains why **Logistic Regression is actually a very simple Neural Network!**

**Mathematical expression of the algorithm**:

For one example $x^{(i)}$:
$$z^{(i)} = w^T x^{(i)} + b \tag{1}$$
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})\tag{2}$$
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})\tag{3}$$

The cost is then computed by summing over all training examples:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})\tag{4}$$

**Key steps**:
In this exercise, you will carry out the following steps: 
- Initialize the parameters of the model
- Learn the parameters for the model by minimizing the cost  
- Use the learned parameters to make predictions (on the test set)
- Analyse the results and conclude

<a name='4'></a>
## 4 - Building the parts of our algorithm

The main steps for building a Neural Network are:
1. Define the model structure (such as number of input features) 
2. Initialize the model's parameters
3. Loop:
    - Calculate current loss (forward propagation)
    - Calculate current gradient (backward propagation)
    - Update parameters (gradient descent)

You often build 1-3 separately and integrate them into one function we call `model()`.

<a name='4-1'></a>
### 4.1 - Helper functions

<a name='ex-3'></a>
### Exercise 3 - sigmoid
Implement `sigmoid()`. You need to compute $sigmoid(z) = \frac{1}{1 + e^{-z}}$ for $z = w^T x + b$ to make predictions. Use np.exp().

In [ ]:
# GRADED FUNCTION: sigmoid

def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """

    #(≈ 1 line of code)
    # s = ...
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE
    
    return s

In [ ]:
print("sigmoid([0, 2]) = " + str(sigmoid(np.array([0, 2]))))

sigmoid_test(sigmoid)

In [ ]:
x = np.array([0.5, 0, 2.0])
output = sigmoid(x)
print(output)

<a name='4-2'></a>
### 4.2 - Initializing parameters

<a name='ex-4'></a>
### Exercise 4 - initialize_with_zeros
Implement parameter initialization in the cell below. You have to initialize w as a vector of zeros. If you don't know what numpy function to use, look up np.zeros() in the Numpy library's documentation.

In [ ]:
# GRADED FUNCTION: initialize_with_zeros

def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias) of type float
    """
    
    # (≈ 2 lines of code)
    # w = ...
    # b = ...
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    return w, b

In [ ]:
dim = 2
w, b = initialize_with_zeros(dim)

assert type(b) == float
print("w = " + str(w))
print("b = " + str(b))

initialize_with_zeros_test_1(initialize_with_zeros)
initialize_with_zeros_test_2(initialize_with_zeros)

<a name='4-3'></a>
### 4.3 - Forward and Backward propagation

Now that your parameters are initialized, you can do the "forward" and "backward" propagation steps for learning the parameters.

<a name='ex-5'></a>
### Exercise 5 - propagate
Implement a function `propagate()` that computes the cost function and its gradient.

**Hints**:

Forward Propagation:
- You get X
- You compute $A = \sigma(w^T X + b) = (a^{(1)}, a^{(2)}, ..., a^{(m-1)}, a^{(m)})$
- You calculate the cost function: $J = -\frac{1}{m}\sum_{i=1}^{m}(y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)}))$

Here are the two formulas you will be using: 

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{7}$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{8}$$

In [ ]:
# GRADED FUNCTION: propagate

def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (n_mels * time_steps, 1)
    b -- bias, a scalar
    X -- data of size (n_mels * time_steps, number of examples)
    Y -- true "label" vector (containing 0 if music, 1 if speech) of size (1, number of examples)

    Return:
    grads -- dictionary containing the gradients of the weights and bias
            (dw -- gradient of the loss with respect to w, thus same shape as w)
            (db -- gradient of the loss with respect to b, thus same shape as b)
    cost -- negative log-likelihood cost for logistic regression
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    #(≈ 2 lines of code)
    # compute activation
    # A = ...
    # compute cost by using np.dot to perform multiplication. 
    # And don't use loops for the sum.
    # cost = ...                                
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    # BACKWARD PROPAGATION (TO FIND GRAD)
    #(≈ 2 lines of code)
    # dw = ...
    # db = ...
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE
    cost = np.squeeze(np.array(cost))

    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [ ]:
w = np.array([[1.], [2]])
b = 1.5
X = np.array([[1., -2., -1.], [3., 0.5, -3.2]])
Y = np.array([[1, 1, 0]])
grads, cost = propagate(w, b, X, Y)

assert type(grads["dw"]) == np.ndarray
assert grads["dw"].shape == (2, 1)
assert type(grads["db"]) == np.float64

print("dw = " + str(grads["dw"]))
print("db = " + str(grads["db"]))
print("cost = " + str(cost))

propagate_test(propagate)

**Expected output**

```
dw = [[ 0.25071532]
 [-0.06604096]]
db = -0.1250040450043965
cost = 0.15900537707692405
```

<a name='4-4'></a>
### 4.4 - Optimization
- You have initialized your parameters.
- You are also able to compute a cost function and its gradient.
- Now, you want to update the parameters using gradient descent.

<a name='ex-6'></a>
### Exercise 6 - optimize
Write down the optimization function. The goal is to learn $w$ and $b$ by minimizing the cost function $J$. For a parameter $\theta$, the update rule is $ \theta = \theta - \alpha \text{ } d\theta$, where $\alpha$ is the learning rate.

In [ ]:
# GRADED FUNCTION: optimize

def optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (n_mels * time_steps, 1)
    b -- bias, a scalar
    X -- data of shape (n_mels * time_steps, number of examples)
    Y -- true "label" vector (containing 0 if music, 1 if speech), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    
    costs = []
    
    for i in range(num_iterations):
        # (≈ 1 lines of code)
        # Cost and gradient calculation 
        # grads, cost = ...
        # YOUR CODE STARTS HERE

        # YOUR CODE ENDS HERE
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        # w = ...
        # b = ...
        # YOUR CODE STARTS HERE

        # YOUR CODE ENDS HERE
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
            # Print the cost every 100 training iterations
            if print_cost:
                print("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
params, grads, costs = optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False)

print("w = " + str(params["w"]))
print("b = " + str(params["b"]))
print("dw = " + str(grads["dw"]))
print("db = " + str(grads["db"]))
print("Costs = " + str(costs))

optimize_test(optimize)

<a name='ex-7'></a>
### Exercise 7 - predict
The previous function will output the learned w and b. We are able to use w and b to predict the labels for a dataset X. Implement the `predict()` function. There are two steps to computing predictions:

1. Calculate $\hat{Y} = A = \sigma(w^T X + b)$

2. Convert the entries of a into 0 (if activation <= 0.5) or 1 (if activation > 0.5), stores the predictions in a vector `Y_prediction`. If you wish, you can use an `if`/`else` statement in a `for` loop (though there is also a way to vectorize this).

In [ ]:
# GRADED FUNCTION: predict

def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (n_mels * time_steps, 1)
    b -- bias, a scalar
    X -- data of size (n_mels * time_steps, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities
    #(≈ 1 line of code)
    # A = ...
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE
    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        #(≈ 4 lines of code)
        # if A[0, i] > ____ :
        #     Y_prediction[0,i] = 
        # else:
        #     Y_prediction[0,i] = 
        # YOUR CODE STARTS HERE

        # YOUR CODE ENDS HERE
    
    return Y_prediction

In [ ]:
w = np.array([[0.1124579], [0.23106775]])
b = -0.3
X = np.array([[1., -1.1, -3.2],[1.2, 2., 0.1]])
print("predictions = " + str(predict(w, b, X)))

predict_test(predict)

<font color='cyan'>

**What to remember:**

You've implemented several functions that:
- Initialize (w,b)
- Optimize the loss iteratively to learn parameters (w,b):
    - Computing the cost and its gradient 
    - Updating the parameters using gradient descent
- Use the learned (w,b) to predict the labels for a given set of examples

<a name='5'></a>
## 5 - Merge all functions into a model

You will now see how the overall model is structured by putting together all the building blocks (functions implemented in the previous parts) together, in the right order.

<a name='ex-8'></a>
### Exercise 8 - model
Implement the model function. Use the following notation:
- Y_prediction_test for your predictions on the test set
- Y_prediction_train for your predictions on the train set
- parameters, grads, costs for the outputs of optimize()

In [ ]:
# GRADED FUNCTION: model

def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (n_mels * time_steps, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (n_mels * time_steps, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to True to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    # (≈ 1 line of code)   
    # initialize parameters with zeros
    # w, b = ...
    
    #(≈ 1 line of code)
    # Gradient descent 
    # params, grads, costs = ...
    
    # Retrieve parameters w and b from dictionary "params"
    # w = ...
    # b = ...
    
    # Predict test/train set examples (≈ 2 lines of code)
    # Y_prediction_test = ...
    # Y_prediction_train = ...
    
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    # Print train/test Errors
    if print_cost:
        print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
from public_tests import *

model_test(model)

If you pass all the tests, run the following cell to train your model.

In [ ]:
logistic_regression_model = model(train_set_x, train_set_y, test_set_x, test_set_y, 
                                   num_iterations=2000, learning_rate=0.0001, print_cost=True)

**Comment**: Analyze your training and test accuracy. The model's performance will depend on the quality and quantity of your audio data. You may see overfitting if the training accuracy is much higher than test accuracy.

You can examine misclassified examples:

In [ ]:
# Example of a misclassified audio clip
index = 0  # Change this to examine different test examples
if index < test_set_x.shape[1]:
    # Reshape back to spectrogram
    spec = test_set_x[:, index].reshape((n_mels, time_steps))
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spec, x_axis='time', y_axis='mel', sr=22050, hop_length=512)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-Spectrogram')
    predicted_label = int(logistic_regression_model['Y_prediction_test'][0, index])
    true_label = int(test_set_y[0, index])
    print("y = " + str(true_label) + ", you predicted that it is a \"" + 
          classes[predicted_label].decode("utf-8") + "\" audio clip.")
    plt.show()

Let's also plot the cost function and the gradients.

In [ ]:
# Plot learning curve (with costs)
costs = np.squeeze(logistic_regression_model['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(logistic_regression_model["learning_rate"]))
plt.show()

**Interpretation**:
You can see the cost decreasing. It shows that the parameters are being learned. However, you see that you could train the model even more on the training set. Try to increase the number of iterations and rerun the cells. You might see that the training set accuracy goes up, but the test set accuracy goes down. This is called overfitting.

<a name='6'></a>
## 6 - Further analysis

Congratulations on building your first audio classification model! Let's analyze it further, and examine possible choices for the learning rate $\alpha$.

#### Choice of learning rate

**Reminder**:
In order for Gradient Descent to work you must choose the learning rate wisely. The learning rate $\alpha$ determines how rapidly we update the parameters. If the learning rate is too large we may "overshoot" the optimal value. Similarly, if it is too small we will need too many iterations to converge to the best values. That's why it is crucial to use a well-tuned learning rate.

Let's compare the learning curve of our model with several choices of learning rates. Run the cell below. This should take about 1 minute.

In [ ]:
learning_rates = [0.01, 0.001, 0.0001]
models = {}

for lr in learning_rates:
    print("Training a model with learning rate: " + str(lr))
    models[str(lr)] = model(train_set_x, train_set_y, test_set_x, test_set_y, 
                           num_iterations=1500, learning_rate=lr, print_cost=False)
    print('\n' + "-------------------------------------------------------" + '\n')

for lr in learning_rates:
    plt.plot(np.squeeze(models[str(lr)]["costs"]), label=str(models[str(lr)]["learning_rate"]))

plt.ylabel('cost')
plt.xlabel('iterations (hundreds)')

legend = plt.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
plt.show()

**Interpretation**: 
- Different learning rates give different costs and thus different predictions results.
- If the learning rate is too large (0.01), the cost may oscillate up and down. It may even diverge (though in this example, using 0.01 still eventually ends up at a good value for the cost). 
- A lower cost doesn't mean a better model. You have to check if there is possibly overfitting. It happens when the training accuracy is a lot higher than the test accuracy.
- In deep learning, we usually recommend that you: 
    - Choose the learning rate that better minimizes the cost function.
    - If your model overfits, use other techniques to reduce overfitting.

<a name='7'></a>
## 7 - Test with your own audio

Congratulations on finishing this assignment! You can use your own audio and see the output of your model. To do that:
1. Add your audio file (.wav format) to the data folder
2. Change the filename in the code below
3. Run the code and check if the algorithm is right (1 = speech, 0 = music)!

In [ ]:
# Test with your own audio file
from audio_utils import load_audio_file
from IPython.display import Audio

# Change this to the name of your audio file
my_audio = "data/test/speech/my_voice_sample.wav"  # Change this path

# Load and preprocess the audio
mel_spec = load_audio_file(my_audio, sr=22050, duration=3.0, n_mels=n_mels)

# Display the spectrogram
plt.figure(figsize=(10, 4))
librosa.display.specshow(mel_spec, x_axis='time', y_axis='mel', sr=22050, hop_length=512)
plt.colorbar(format='%+2.0f dB')
plt.title('Your Audio - Mel-Spectrogram')
plt.show()

# Play the audio
print("Listen to the audio:")
display(Audio(my_audio))

# Flatten and predict
audio_vector = mel_spec.reshape(-1, 1)
my_predicted_audio = predict(logistic_regression_model["w"], logistic_regression_model["b"], audio_vector)

print("\ny = " + str(int(np.squeeze(my_predicted_audio))) + 
      ", your algorithm predicts a \"" + classes[int(np.squeeze(my_predicted_audio))].decode("utf-8") + "\" audio clip.")

<font color='cyan'>

**What to remember from this assignment:**
1. Preprocessing the dataset is important (converting audio to spectrograms).
2. You implemented each function separately: initialize(), propagate(), optimize(). Then you built a model().
3. Tuning the learning rate (which is an example of a "hyperparameter") can make a big difference to the algorithm.
4. Audio classification follows the same principles as image classification when using appropriate representations (spectrograms).

**Things to try:**
- Play with the learning rate and the number of iterations
- Try different audio datasets (voice vs non-voice, male vs female, etc.)
- Experiment with different spectrogram parameters (n_mels, duration)
- Collect your own audio samples and train on them!